In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import manifold
import umap
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris

In [ ]:
# Load and preprocess Iris dataset
n_gauss = 3  # Equal to number of classes
dim = 3  # 4 is maximum dimension

iris = load_iris()
X = iris.data[:, :dim]  # Use only the first two features
# X = iris.data  # Use only the first two features
D = StandardScaler().fit_transform(X)  # Standardize features

c = iris.target  # Class labels

class_label= ['setosa', 'versicolor', 'virginica']


D.shape

In [ ]:
%matplotlib qt

# colors = ['r', 'g', 'b']  # Red, Green, Blue
colors = ['#FF0000', '#00FF00', '#0000FF', '#FFFF00', '#00FFFF', '#FF00FF', '#000000']
# Create a figure and 3D axis
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(projection='3d')

# Define colors for each Gaussian distribution

# Loop through each Gaussian to plot points with corresponding color
for i in range(n_gauss):
    ax.scatter(D[c == i, 0], D[c == i, 1], D[c == i, 2], color=colors[i], label=class_label[i])
    # ax.scatter(D[:,0], D[:,1], D[:,2], c=c)

# Set labels and title
ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')
ax.set_zlabel('Z-axis')
ax.set_title('3D Scatter Plot of Data Points from Three Gaussian Distributions')

# Add a legend
ax.legend()

# Show the plot
plt.show()

In [ ]:
t_sne = umap.UMAP(
    n_components=2,     # Targeting 2D projection
    n_neighbors=10,     # Similar to t-SNE perplexity
    min_dist=0.1,       # Controls the compactness of the clusters
    init="random",
    random_state=0,
)


# Apply UMAP on the 3D Gaussian data `D`
S = t_sne.fit_transform(D)

In [ ]:
# Plotting the t-SNE results with the same color scheme
%matplotlib qt

# colors = ['r', 'g', 'b','']  # Red, Green, Blue
plt.figure(figsize=(10, 8))
for i in range(n_gauss):
    plt.scatter(S[c == i, 0], S[c == i, 1], color=colors[i], label=class_label[i])
    # plt.scatter(S[c == i, 0], S[c == i, 1], label=f'Gaussian {i+1}')

plt.title('t-SNE Visualization of 3D Gaussian Distributions into 2D')
plt.legend()
plt.grid(True)
plt.show()

## Create a 2D Grid for Jacobian Calculation

In [ ]:
# Define min and max values
x_min, x_max = np.min(S[:, 0]), np.max(S[:, 0])
y_min, y_max = np.min(S[:, 1]), np.max(S[:, 1])
print(x_min, x_max)
print(y_min, y_max)

In [ ]:
# Define grid resolution
num_grid_points = 100

# Generate grid
x_vals = np.linspace(x_min, x_max, num_grid_points)
y_vals = np.linspace(y_min, y_max, num_grid_points)
xx, yy = np.meshgrid(x_vals, y_vals)
print(yy.shape)
print(y_vals.shape)

In [ ]:
%matplotlib qt

plt.figure(figsize=(10, 8))
# Visualize the grid on top of the t-SNE data
plt.scatter(S[:, 0], S[:, 1], c='blue', s=10, label="t-SNE Output")
plt.scatter(xx, yy, c='red', s=5, label="Grid Points")
plt.title("2D t-SNE Output with Grid Points")
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
# plt.grid(True)
plt.show()

## Define the Inverse Projection

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from sklearn.model_selection import train_test_split

# Define the MLP inverse_model
class NNinv(nn.Module):
    def __init__(self, input_size, output_size):
        super(NNinv, self).__init__()
        
        # Define the layers
        self.layers = nn.Sequential(
            nn.Linear(input_size, 64),  # Input to first hidden layer
            nn.ReLU(),
            nn.Linear(64, 128),  # First hidden layer to second hidden layer
            nn.ReLU(),
            nn.Linear(128, 256),  # Second hidden layer to third hidden layer
            nn.ReLU(),
            nn.Linear(256, 512),  # Third hidden layer to fourth hidden layer
            nn.ReLU(),
            nn.Linear(512, output_size),  # Fifth hidden layer to output
            nn.Sigmoid()  # Output layer with sigmoid activation
        )
    
    def forward(self, x):
        return self.layers(x)


In [ ]:
X_train, X_test, y_train, y_test, c_train, c_test = train_test_split(S, D,c, test_size=0.33, random_state=42, stratify=c)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(c_train.shape)
print(c_test.shape)

In [ ]:
# import numpy as np

# Assuming c_train is your array
unique_values, counts = np.unique(c_test, return_counts=True)

# Display the unique values and their counts
for value, count in zip(unique_values, counts):
    print(f"Class {value}: {count} occurrences")


## Model Training

In [ ]:
# Example usage
input_size = 2  # Example input size (can be changed)
output_size = dim   # Binary classification (sigmoid output for single output)

# Create DataLoader for batch processing
batch_size = 64
t_X_train = torch.tensor(X_train)
t_y_train = torch.tensor(y_train)
dataset = TensorDataset(t_X_train, t_y_train)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Instantiate the inverse_model, loss function, and optimizer
inverse_model = NNinv(input_size, output_size)
loss_fn = nn.L1Loss()  # Mean Absolute Error (MAE)
optimizer = optim.Adam(inverse_model.parameters(), lr=0.001)

# Number of epochs to train
num_epochs = 2000
# num_epochs = 5

# Training loop
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (inputs, targets) in enumerate(dataloader):
        # Forward pass
        outputs = inverse_model(inputs)
        loss = loss_fn(outputs, targets)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    # Print the average loss for the epoch
    avg_loss = running_loss / len(dataloader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

print("Training complete.")

In [ ]:
t_X_test = torch.tensor(X_test)
t_y_test = torch.tensor(y_test)
outputs_test = inverse_model(t_X_test)
loss_test = loss_fn(outputs_test, t_y_test)
print(loss_test/y_test.shape[0])

# Visualizing Inverse Projection

In [ ]:
%matplotlib qt

# Create a figure and 3D axis
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(projection='3d')

# Define colors for each Gaussian distribution
# colors = ['r', 'g', 'b']  # Red, Green, Blue


output_fin = outputs_test.detach().numpy()
# Loop through each Gaussian to plot points with corresponding color
for i in range(n_gauss):
    ax.scatter(t_y_test[c_test == i, 0], t_y_test[c_test == i, 1], t_y_test[c_test == i, 2], color=colors[i], label=f'Actual_Gaussian {i+1}')
    # ax.scatter(output_fin[c_test == i, 0], output_fin[c_test == i, 1], output_fin[c_test == i, 2], color='orange', label=f'Predicted_Gaussian {i+1}')

ax.scatter(output_fin[:, 0], output_fin[:, 1], output_fin[:, 2], color='orange', label=f'Predicted_Gaussians')

# Set labels and title
ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')
ax.set_zlabel('Z-axis')
ax.set_title('TSNE \n Actual Vs Prediction')

# Add a legend
ax.legend()

# Show the plot
plt.show()

## Validating 2D projection


In [ ]:
def generate_spread_points(S, labels, num_new_points_per_cluster=5, spread_factor=0.5):
    """
    Generate new points around the spread of the Gaussian clusters in 2D t-SNE space.
    
    Parameters:
    S (np.array): 2D t-SNE points (original).
    labels (np.array): Labels for the original points, corresponding to Gaussian clusters.
    num_new_points_per_cluster (int): Number of new points to generate per Gaussian cluster.
    spread_factor (float): Spread factor controlling the variance of new points.
    
    Returns:
    new_points (np.array): Newly generated points spread around each cluster.
    new_labels (np.array): Labels corresponding to the new points.
    """
    new_points = []
    new_labels = []
    
    # Get the unique labels (each label corresponds to one Gaussian)
    unique_labels = np.unique(labels)

    for label in unique_labels:
        # Get the points that belong to the current Gaussian cluster
        cluster_points = S[labels == label]
        
        # Calculate covariance matrix for the current cluster
        cluster_cov = np.cov(cluster_points.T)

        for _ in range(num_new_points_per_cluster):
            # Randomly choose a point within the cluster
            random_point = cluster_points[np.random.randint(len(cluster_points))]
            
            # Generate a random offset using the covariance matrix to create a spread
            offset = np.random.multivariate_normal([0, 0], spread_factor * cluster_cov)

            # Add the offset to the selected random point to create a new point
            new_point = random_point + offset
            new_points.append(new_point)
            new_labels.append(label)  # Assign the same label as the original points
    
    return np.array(new_points), np.array(new_labels)


In [ ]:
new_points, new_labels = generate_spread_points(S, c, num_new_points_per_cluster=20, spread_factor=0.3)

In [ ]:
markers = ['$G1$', '$G2$', '$G3$', '$G4$', '$G5$', '$G6$', '$G7$']

plt.figure(figsize=(10, 8))
for i in range(n_gauss):
    plt.scatter(S[c == i, 0], S[c == i, 1], color=colors[i], label=f'Gaussian {i+1}')

    # Plot new points
    plt.scatter(new_points[new_labels == i, 0], new_points[new_labels == i, 1], color=colors[i], marker = markers[i] , s = 100, edgecolors='black', label= f'New Points_Gaussian {i+1}')

# plt.scatter(new_points[:, 0], new_points[:, 1], color='brown', label="New Points")

plt.legend()
plt.title("Original and Generated Points in 2D t-SNE Space")
plt.show()


# Apply train model on new points

In [ ]:
new_points_test = torch.tensor(new_points).float()
outputs_new_points = inverse_model(new_points_test)
outputs_new_points =outputs_new_points.detach().numpy()

# Visualize

In [ ]:
%matplotlib qt

# Create a figure and 3D axis
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(projection='3d')

# # Define colors for each Gaussian distribution
# colors = ['r', 'g', 'b']  # Red, Green, Blue

# Loop through each Gaussian to plot points with corresponding color
for i in range(n_gauss):
    ax.scatter(D[c == i, 0], D[c == i, 1], D[c == i, 2], color=colors[i], alpha=0.7, label=f'Gaussian {i+1}')
    ax.scatter(outputs_new_points[new_labels == i, 0], outputs_new_points[new_labels == i, 1], outputs_new_points[new_labels == i, 2], marker=markers[i],alpha=1.0, s=150, edgecolors='black', label=f'New_points_Gaussian {i+1}')

# ax.scatter(outputs_new_points[:, 0], outputs_new_points[:, 1], outputs_new_points[:, 2], color='k', label=f'Predicted_Gaussian {i+1}')

# Set labels and title
ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')
ax.set_zlabel('Z-axis')
ax.set_title(' New points (2D TSNE) mapping into 3D Gaussian Distributions')

# Add a legend
ax.legend()

# Show the plot
plt.show()

# Define Jacobian

In [ ]:
def compute_jacobian(x, y):
    """
    Computes the Jacobian matrix for each point in the grid.
    
    Args:
        grid_points (ndarray): A 2D array of shape (n_points, 2) representing the grid points.

    Returns:
        jacobian_matrices (list): A list of jacobian matrices for each grid point.
    """
    jacobian_matrices = []
    
    # Define the model's forward pass to use autograd
    def model_forward(input):
        return inverse_model(input)  # Model's forward pass
    
    # Iterate through the grid points
    # for point in grid_points:
    point_tensor = torch.tensor([x, y], dtype=torch.float32, requires_grad=True)  # (1, 2) tensor
    
    # Compute the Jacobian using autograd's jacobian function
    jacobian = torch.autograd.functional.jacobian(model_forward, point_tensor)
    
        # The output of jacobian will have shape (1, 3, 2), so we need to squeeze to get (3, 2)
        # jacobian_matrices.append(jacobian.squeeze(0))  # Remove the batch dimension
    
    return jacobian

def compute_jacobian_implement(x, y, eps=1e-5):
    # Create tensor point for cloning
    point = torch.tensor([[x, y]], dtype=torch.float32)

    # Partial derivative w.r.t. x using five-point stencil
    f_x_2plus = inverse_model(torch.tensor([[x + 2 * eps, y]], dtype=torch.float32))
    f_x_plus = inverse_model(torch.tensor([[x + eps, y]], dtype=torch.float32))
    f_x_minus = inverse_model(torch.tensor([[x - eps, y]], dtype=torch.float32))
    f_x_2minus = inverse_model(torch.tensor([[x - 2 * eps, y]], dtype=torch.float32))
    
    df_dx = (-f_x_2plus + 8 * f_x_plus - 8 * f_x_minus + f_x_2minus) / (12 * eps)

    # Partial derivative w.r.t. y using five-point stencil
    f_y_2plus = inverse_model(torch.tensor([[x, y + 2 * eps]], dtype=torch.float32))
    f_y_plus = inverse_model(torch.tensor([[x, y + eps]], dtype=torch.float32))
    f_y_minus = inverse_model(torch.tensor([[x, y - eps]], dtype=torch.float32))
    f_y_2minus = inverse_model(torch.tensor([[x, y - 2 * eps]], dtype=torch.float32))
    
    df_dy = (-f_y_2plus + 8 * f_y_plus - 8 * f_y_minus + f_y_2minus) / (12 * eps)

    # Stack results to form Jacobian matrix
    jacobian = torch.stack([df_dx.squeeze(), df_dy.squeeze()], dim=1)
    
    return jacobian.detach().numpy()

In [ ]:
# Calculate Jacobians over the grid and store results
jacobians = []
for i in range(num_grid_points):
    for j in range(num_grid_points):
        x, y = xx[i, j], yy[i, j]
        # print(x,y)
        jacobian = compute_jacobian(x, y)
        # jacobian = compute_jacobian_implement(x, y, 1e-5)
        # print(jacobian)
        jacobians.append(jacobian)

## Reshaping Jacobian [num_girds, num_grids, output_dim, input_dim]

In [ ]:

# Convert the list of numpy arrays into a list of PyTorch tensors
jacobian_tensors = [torch.tensor(jacob) for jacob in jacobians]

# Convert the list into a 3D tensor
jacobian_tensor = torch.stack(jacobian_tensors)  # Shape will be [num_grids * num_grids, out, inp]

# Reshape the tensor to [num_grids, num_grids, 3, 2]
jacobian_tensor_reshaped = jacobian_tensor.view(num_grid_points, num_grid_points, dim, 2)
jacobian_tensor_reshaped.shape

# SVD

In [ ]:
# Initialize lists to store singular values, U and V matrices for each Jacobian
singular_values_list = []
U_matrices = []
V_matrices = []

# Iterate over each grid point and apply SVD
for i in range(jacobian_tensor_reshaped.shape[0]):
    for j in range(jacobian_tensor_reshaped.shape[1]):
        jacobian_matrix = jacobian_tensor_reshaped[i, j]  # Shape: (3, 2)
        
        
        # Perform SVD
        U, SV, Vt = torch.linalg.svd(jacobian_matrix, full_matrices=False)
        # print(SV)
        # np.savetxt('jacob_matrix'+ str(i),jacobian_matrix)
    
        # break
        # Store the singular values and U, V matrices
        singular_values_list.append(SV)
        U_matrices.append(U)
        V_matrices.append(Vt)

In [ ]:
print(U_matrices[0].shape)
print(singular_values_list[0].shape)
print(V_matrices[0].shape)

print(len(U_matrices))
print(len(singular_values_list))
print(len(V_matrices))

In [ ]:
# # Define min and max values
# x_min, x_max = np.min(S[:, 0]), np.max(S[:, 0])
# y_min, y_max = np.min(S[:, 1]), np.max(S[:, 1])
# print(x_min, x_max)
# print(y_min, y_max)
import matplotlib.pyplot as plt
import numpy as np

# Reshape singular values to the grid's shape for visualization
# Extract the largest singular value for each point in the grid
largest_singular_values = np.array([s[0] for s in singular_values_list]).reshape(num_grid_points, num_grid_points)


# Plot heatmap
plt.figure(figsize=(8, 6))
plt.imshow(largest_singular_values,
           extent=(x_min, x_max, y_min, y_max),
           origin='lower', 
           cmap='hot', 
           interpolation='nearest')
plt.colorbar(label="Largest Singular Value")
plt.xlabel("Grid X")
plt.ylabel("Grid Y")
plt.title("Heatmap of Jacobian Singular Values")
plt.show()

# Overlayplot

In [ ]:
# Define min and max values
x_min, x_max = np.min(S[:, 0]), np.max(S[:, 0])
y_min, y_max = np.min(S[:, 1]), np.max(S[:, 1])
print(x_min, x_max)
print(y_min, y_max)

# Define grid resolution 
num_grid_points = 100

# Generate grid
x_vals = np.linspace(x_min, x_max, num_grid_points)
y_vals = np.linspace(y_min, y_max, num_grid_points)
xx, yy = np.meshgrid(x_vals, y_vals)
grid_points = np.c_[xx.ravel(), yy.ravel()]

jacobian_norms = np.zeros(len(grid_points))
for idx, point in enumerate(grid_points):
    point_tensor = torch.tensor(point, dtype=torch.float32, requires_grad=True).view(1, 2)
    
    # Compute the Jacobian for the current point
    jacobian = torch.autograd.functional.jacobian(lambda x: inverse_model(x), point_tensor)
    
    # Reshape Jacobian to 2D: (output_dim, input_dim)
    jacobian_2d = jacobian.view(dim, 2)  # Assuming output is (1, 3), input is (1, 2)
    
    # Compute spectral norm (largest singular value)
    jacobian_norms[idx] = torch.linalg.norm(jacobian_2d, ord=2).item()

jacobian_norms = jacobian_norms.reshape(xx.shape)

# Step 4: Plot heatmap with t-SNE points overlayed
plt.figure(figsize=(10, 8))

# Plot heatmap
plt.imshow(
    jacobian_norms,
    extent=(x_min, x_max, y_min, y_max),
    origin='lower',
    cmap='hot',
    alpha=0.6
)
plt.colorbar(label='Spectral Norm of Jacobian')

# Overlay t-SNE points
# plt.scatter(S[:, 0], S[:, 1], c='blue', edgecolor='k', label='t-SNE points')

for i in range(n_gauss):
    plt.scatter(S[c == i, 0], S[c == i, 1], color=colors[i], label=class_label[i])


# Labels and title
plt.title("Overlaying t-SNE points on Jacobian Heatmap")
plt.xlabel("t-SNE Dimension 1")
plt.ylabel("t-SNE Dimension 2")
plt.legend()
plt.show()


# Jacobian estiamtion Analytically SYmbolically (IRIS Dataset)

## Load IRIS Dataset

In [ ]:
# Load and preprocess Iris dataset
n_gauss = 3  # Equal to number of classes
dim = 3  # 4 is maximum dimension

iris = load_iris()
X = iris.data[:, :dim]  # Use only the first two features
# X = iris.data  # Use only the first two features
D = StandardScaler().fit_transform(X)  # Standardize features

c = iris.target  # Class labels

class_label= ['setosa', 'versicolor', 'virginica']


D.shape

## Define Analytic, Symbolic, Autograd, our implementation for Jacobian

In [ ]:
# New analytic function
def analytic_function(point):
    x, y = point[:, 0], point[:, 1]
    z1 = x**2 * y + torch.sin(x * y)
    z2 = y**2 * x + torch.cos(x * y)
    return torch.stack([z1, z2], dim=1)

# Symbolic Jacobian
def symbolic_jacobian(x, y):
    return np.array([
        [2 * x * y + y * np.cos(x * y), x**2 + x * np.cos(x * y)],
        [y**2 + y * np.cos(x * y), 2 * y * x - x * np.sin(x * y)],
    ])


# Numerical Jacobian using autograd
def compute_jacobian_autograd(func, point):
    point_tensor = torch.tensor([point], dtype=torch.float32, requires_grad=True)
    jacobian = torch.autograd.functional.jacobian(func, point_tensor)
    return jacobian.squeeze(0).squeeze(1).detach().numpy()

# Five-point stencil for numerical Jacobian
def compute_jacobian_implement(x, y, eps=1e-5):
    # Compute partial derivatives using five-point stencil
    f_x_2plus = analytic_function(torch.tensor([[x + 2 * eps, y]], dtype=torch.float32))
    f_x_plus = analytic_function(torch.tensor([[x + eps, y]], dtype=torch.float32))
    f_x_minus = analytic_function(torch.tensor([[x - eps, y]], dtype=torch.float32))
    f_x_2minus = analytic_function(torch.tensor([[x - 2 * eps, y]], dtype=torch.float32))
    df_dx = (-f_x_2plus + 8 * f_x_plus - 8 * f_x_minus + f_x_2minus) / (12 * eps)

    f_y_2plus = analytic_function(torch.tensor([[x, y + 2 * eps]], dtype=torch.float32))
    f_y_plus = analytic_function(torch.tensor([[x, y + eps]], dtype=torch.float32))
    f_y_minus = analytic_function(torch.tensor([[x, y - eps]], dtype=torch.float32))
    f_y_2minus = analytic_function(torch.tensor([[x, y - 2 * eps]], dtype=torch.float32))
    df_dy = (-f_y_2plus + 8 * f_y_plus - 8 * f_y_minus + f_y_2minus) / (12 * eps)

    return torch.stack([df_dx.squeeze(), df_dy.squeeze()], dim=1).detach().numpy()


## Compute Spectral Norm

In [ ]:
spectral_norm_symbolic = np.zeros(X.shape[0])
spectral_norm_autograd = np.zeros_like(spectral_norm_symbolic)
spectral_norm_implement = np.zeros_like(spectral_norm_symbolic)

for i in range(X.shape[0]):
    symbolic_J = jacobian_symbolic[i]
    autograd_J = jacobian_autograd[i]
    implement_J = jacobian_implement[i]

    spectral_norm_symbolic[i] = np.linalg.norm(symbolic_J, ord=2)
    spectral_norm_autograd[i] = np.linalg.norm(autograd_J, ord=2)
    spectral_norm_implement[i] = np.linalg.norm(implement_J, ord=2)


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from matplotlib.colors import ListedColormap
from scipy.interpolate import griddata

# Load and preprocess Iris dataset
iris = load_iris()
X = iris.data[:, :2]  # Use only the first two features
y = iris.target  # Target labels
X = StandardScaler().fit_transform(X)  # Standardize features

# Define an analytic function for transformation
def analytic_function(point):
    x, y = point[:, 0], point[:, 1]
    z1 = x**2 * y + torch.sin(x * y)
    z2 = y**2 * x + torch.cos(x * y)
    return torch.stack([z1, z2], dim=1)

# Symbolic Jacobian
def symbolic_jacobian(x, y):
    return np.array([
        [2 * x * y + y * np.cos(x * y), x**2 + x * np.cos(x * y)],
        [y**2 + y * np.cos(x * y), 2 * y * x - x * np.sin(x * y)],
    ])

# Numerical Jacobian using autograd
def compute_jacobian_autograd(func, point):
    point_tensor = torch.tensor([point], dtype=torch.float32, requires_grad=True)
    jacobian = torch.autograd.functional.jacobian(func, point_tensor)
    return jacobian.squeeze(0).squeeze(1).detach().numpy()

# Numerical Jacobian using five-point stencil
def compute_jacobian_implement(x, y, eps=1e-5):
    f_x_2plus = analytic_function(torch.tensor([[x + 2 * eps, y]], dtype=torch.float32))
    f_x_plus = analytic_function(torch.tensor([[x + eps, y]], dtype=torch.float32))
    f_x_minus = analytic_function(torch.tensor([[x - eps, y]], dtype=torch.float32))
    f_x_2minus = analytic_function(torch.tensor([[x - 2 * eps, y]], dtype=torch.float32))
    df_dx = (-f_x_2plus + 8 * f_x_plus - 8 * f_x_minus + f_x_2minus) / (12 * eps)

    f_y_2plus = analytic_function(torch.tensor([[x, y + 2 * eps]], dtype=torch.float32))
    f_y_plus = analytic_function(torch.tensor([[x, y + eps]], dtype=torch.float32))
    f_y_minus = analytic_function(torch.tensor([[x, y - eps]], dtype=torch.float32))
    f_y_2minus = analytic_function(torch.tensor([[x, y - 2 * eps]], dtype=torch.float32))
    df_dy = (-f_y_2plus + 8 * f_y_plus - 8 * f_y_minus + f_y_2minus) / (12 * eps)

    return torch.stack([df_dx.squeeze(), df_dy.squeeze()], dim=1).detach().numpy()

# Initialize Jacobians and spectral norms
jacobian_symbolic = np.zeros((X.shape[0], 2, 2))
jacobian_autograd = np.zeros_like(jacobian_symbolic)
jacobian_implement = np.zeros_like(jacobian_symbolic)

spectral_norm_symbolic = np.zeros(X.shape[0])
spectral_norm_autograd = np.zeros_like(spectral_norm_symbolic)
spectral_norm_implement = np.zeros_like(spectral_norm_symbolic)

# Compute Jacobians and spectral norms
for i, (x, y) in enumerate(X):
    symbolic_J = symbolic_jacobian(x, y)
    autograd_J = compute_jacobian_autograd(analytic_function, [x, y])
    implement_J = compute_jacobian_implement(x, y, 1e-4)

    jacobian_symbolic[i] = symbolic_J
    jacobian_autograd[i] = autograd_J
    jacobian_implement[i] = implement_J

    spectral_norm_symbolic[i] = np.linalg.norm(symbolic_J, ord=2)
    spectral_norm_autograd[i] = np.linalg.norm(autograd_J, ord=2)
    spectral_norm_implement[i] = np.linalg.norm(implement_J, ord=2)

# Create a grid for feature space
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200), np.linspace(y_min, y_max, 200))

# Interpolate spectral norms onto the grid
def interpolate_spectral_norm(norm_values):
    return griddata(X[:, :2], norm_values, (xx, yy), method='cubic')

spectral_norm_symbolic_interp = interpolate_spectral_norm(spectral_norm_symbolic)
spectral_norm_autograd_interp = interpolate_spectral_norm(spectral_norm_autograd)
spectral_norm_implement_interp = interpolate_spectral_norm(spectral_norm_implement)

# Train a classifier to get decision boundaries
classifier = LogisticRegression()
classifier.fit(X, y)
Z = classifier.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

# Function to plot spectral norm heatmap with decision boundaries
def plot_with_boundaries(spectral_norm_interp, method_name):
    plt.figure(figsize=(8, 6))
    plt.contourf(xx, yy, spectral_norm_interp, levels=20, cmap='coolwarm', alpha=0.8)
    plt.colorbar(label='Spectral Norm')
    plt.contour(xx, yy, Z, levels=np.arange(4) - 0.5, cmap=ListedColormap(['black']), linestyles='--', linewidths=1)
    plt.scatter(X[:, 0], X[:, 1], c=y, edgecolor='k', cmap=ListedColormap(['red', 'green', 'blue']), s=20)
    plt.title(f"Spectral Norm and Decision Boundaries ({method_name})")
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.show()

# Plot results for each method
plot_with_boundaries(spectral_norm_symbolic_interp, "Symbolic")
plot_with_boundaries(spectral_norm_autograd_interp, "Autograd")
plot_with_boundaries(spectral_norm_implement_interp, "Our Implementation")
